---
Инициализация БД для записи данных
---

- Формирование структуры БД

In [1]:
%run -i ..//data/bd_init_script

----> Script was successfully executed
Wall time: 234 ms


- Заполнение статичными словарями

In [1]:
%run -i ..//data/dicts_init_script

----> Into table schedule was inserted 5 values
----> Into table experience was inserted 4 values
----> Into table currency was inserted 10 values
----> Into table employment was inserted 5 values
----> Into table specialization was inserted 621 values


---
Обход ограничения максимальной глубины запроса
---
Для обхода максимальной глубины запроса к API, получаем промежутки времени, в которые кол-во опубликованых вакансий не
превышает 2000. Далее будут получены данные по этим промежуткам.

(Данные только о вакансиях в сфере IT)

In [1]:
%%time
from data_parsers.hhParser import HhParser
from datetime import datetime, timedelta

parser = HhParser()
cur_date = {
    'from': (datetime.now() - timedelta(days=1)),
    'to': datetime.now()
}
req_param = {
        'specialization':1,
        'per_page':0,
        'date_from':cur_date['from'].strftime('%Y-%m-%dT%H:%M:%S'),
        'date_to':cur_date['to'].strftime('%Y-%m-%dT%H:%M:%S')
    }
data = parser.execute_request(req_param=req_param)
dates = []
total_vacancies_number = 0
while data['found'] != 0:
    while data['found'] > 2000:
        cur_date['from'] += timedelta(hours=1)
        req_param['date_from'] = cur_date['from'].strftime('%Y-%m-%dT%H:%M:%S')
        data = parser.execute_request(req_param=req_param)

    total_vacancies_number += data['found']
    dates.append([cur_date['from'].strftime('%Y-%m-%dT%H:%M:%S'), cur_date['to'].strftime('%Y-%m-%dT%H:%M:%S')])

    cur_date['to'] = cur_date['from']
    cur_date['from'] -= timedelta(hours=12)
    req_param['date_from'] = cur_date['from'].strftime('%Y-%m-%dT%H:%M:%S')
    req_param['date_to'] = cur_date['to'].strftime('%Y-%m-%dT%H:%M:%S')
    data = parser.execute_request(req_param=req_param)
print(f'За последний месяц имеется {total_vacancies_number} вакансий')

За последний месяц имеется 119637 вакансий
Wall time: 2min 15s


---
Выгрузка всех вакансий
---
Получение данных по вакансиям за ранее полученные промежутки времени

In [2]:
%%time
from services.db_service import DbService
from tqdm.notebook import tqdm
import threading
import gc

def load_vacancies(d_range):
    params = {
        'page': 0,
        'per_page': 100,
        'date_from': d_range[0],
        'date_to': d_range[1],
        'specialization':1
    }
    vac = parser.get_vacancies(req_params=params)
    db_service.save_vacancies(vac)
    vac.clear()

db_service = DbService()
THREADS_NUMBER = 6
for i in tqdm(range(0, len(dates), THREADS_NUMBER)):

    threads = []
    upper_i = i + THREADS_NUMBER
    if upper_i > len(dates):
        upper_i = len(dates)

    for j in range(i, upper_i):
        threads.append(threading.Thread(target=load_vacancies, args=(dates[j],)))

    for t in threads:
        t.start()

    for t in threads:
        t.join()
    threads.clear()
    gc.collect()

  0%|          | 0/16 [00:00<?, ?it/s]

Получение id вакансий:   0%|          | 0/16 [00:00<?, ?it/s]

Получение id вакансий:   0%|          | 0/14 [00:00<?, ?it/s]

Получение id вакансий:   0%|          | 0/19 [00:00<?, ?it/s]

Получение id вакансий:   0%|          | 0/17 [00:00<?, ?it/s]

...

Сохранение вакансий в БД:   0%|          | 0/117 [00:00<?, ?it/s]

Сохранение вакансий в БД:   0%|          | 0/155 [00:00<?, ?it/s]

Сохранение вакансий в БД:   0%|          | 0/655 [00:00<?, ?it/s]

Сохранение вакансий в БД:   0%|          | 0/1570 [00:00<?, ?it/s]

Сохранение вакансий в БД:   0%|          | 0/1904 [00:00<?, ?it/s]